Total screws detected: 0


Number of items in the image: 62


Total screws detected: 2


In [82]:
import cv2
import numpy as np

def count_screws_final(image_path, display_width=800):
    image = cv2.imread(image_path)
    if image is None:
        print("Image not found.")
        return

    height, width = image.shape[:2]
    ratio = display_width / width

    # Grayscale + CLAHE
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    gray_eq = clahe.apply(gray)

    # Threshold
    thresh = cv2.adaptiveThreshold(
        gray_eq, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, 15, 5
    )

    # Morphological cleanup
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)

    # Contour detection
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    result = image.copy()
    count = 0

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 200 or area > 3000:
            continue  # Filter based on area

        # Solidity = contour area / convex hull area
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        if hull_area == 0:
            continue
        solidity = float(area) / hull_area
        if solidity < 0.6:
            continue  # Discard contours that are not compact

        # Bounding box shape filter
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        if 0.3 < aspect_ratio < 3.0:
            count += 1
            cv2.drawContours(result, [cnt], -1, (0, 255, 0), 1)

    # Display count
    cv2.putText(result, f"Screws Counted: {count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    def resize(img): return cv2.resize(img, (int(width * ratio), int(height * ratio)))
    cv2.imshow("Thresh", resize(thresh))
    cv2.imshow("Morph", resize(morph))
    cv2.imshow("Result", resize(result))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return count

# Run the detector
image_path = "ScrewAndBolt_20240713/20240713_192951.jpg"
print("Detected screws:", count_screws_final(image_path))


Detected screws: 38


In [81]:
Screws_2024_07_15/img1.jpg
ScrewAndBolt_20240713/20240713_192951.jpg

SyntaxError: invalid imaginary literal (3059886166.py, line 2)